In [1]:
%pip install requests pillow tqdm

Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
from PIL import Image
from io import BytesIO
import os
import json
import base64
from pathlib import Path
from tqdm import tqdm

In [3]:
def ensure_directory_exists(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

def encode_image_to_base64(image_path):
    """Encode image to Base64 string."""
    with Image.open(image_path) as image:
        buffered = BytesIO()
        image.save(buffered, format="JPEG")  # You can change the format to PNG if required
        return base64.b64encode(buffered.getvalue()).decode('utf-8')

def send_image(model_url, image_path, version, 
               prompt = 'street', age = '20', bodyShape = 'slim', ethnic = 'asian', sex = 'female', skinColor = 'white'):
    """Sends a JSON payload with image URLs and additional parameters to the API."""
    api_url = ''
    if version == 'released': 
        api_url = 'https://tryon-advanced.tianlong.co.uk/upload/images'
    elif version == 'test': 
        api_url = 'https://tryon-advanced-test.tianlong.co.uk/upload/images'
        
    face_name = Path(image_path).stem
    face_image_base64 = encode_image_to_base64(image_path)
        
    headers = {
        'accept': 'application/json',
        'accept-language': 'en-US,en;q=0.9,zh-CN;q=0.8,zh;q=0.7',
        'content-type': 'application/json',
        'cookie': 'notion_browser_id=002af42f-7cf4-4482-aadf-cb1a963d4333; intercom-id-gpfdrxfd=dae204e3-2158-4794-a9d5-806af9449033; intercom-device-id-gpfdrxfd=48b033f5-8ce4-4d8f-8f30-35af5f8dfff1; amp_af43d4=002af42f7cf44482aadfcb1a963d4333...1hs9k2j74.1hs9k2j75.d.0.d',
        'origin': 'chrome-extension://ommjpdpphbobbcnjkfdmfekajmhdiijf',
        'priority': 'u=1, i',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'none',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36'
    }

    data = {
        'model': model_url,
        'face': f"{face_image_base64}",
        'prompt': prompt,
        'seed': 7894674637868,
        'enhanceTryOnData': {
            'age': age,
            'bodyShape': bodyShape,
            'ethnic': ethnic,
            'sex': sex,
            'skinColor': skinColor
        }
    }
    
    

    response = requests.post(api_url, headers=headers, json=data)
    if response.status_code == 200 and 'application/json' in response.headers.get('Content-Type', ''):
        response_data = response.json()
        if response_data.get('status') == 'success' and 'image' in response_data:
            image_data = base64.b64decode(response_data['image'])
            image = Image.open(BytesIO(image_data))
            generated_image_path = f'./generated_folder/{version}/{face_name}_{version}.png'
            image.save(generated_image_path, format='PNG')
            return generated_image_path
    print(f'Failed to retrieve an image from {version} version.')
    print('Status Code:', response.status_code)
    print('Response:', response.text)
    return None


In [4]:
def create_html(model_url, face_image_path, released_image_path, test_image_path, face_name):
    """Create an HTML file to display images side by side."""
    html_content = f"""
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Image Comparison - {face_name}</title>
        <style>
            .image-row {{
                display: flex;
                justify-content: space-around;
                align-items: center;
                margin-top: 20px;
            }}
            .image-row div {{
                text-align: center;
            }}
            img {{
                max-width: 400px;
                margin: 10px;
                border: 2px solid #ccc;
            }}
        </style>
    </head>
    <body>
        <h1>Comparison for {face_name}</h1>
        <div class="image-row">
            <div>
                <h2>Model Image</h2>
                <img src="{model_url}" alt="Model Image">
            </div>
            <div>
                <h2>Face Image</h2>
                <img src="{face_image_path}" alt="Face Image">
            </div>
        </div>
        <div class="image-row">
            <div>
                <h2>Generated Image (Released Version)</h2>
                <img src="{released_image_path}" alt="Generated Image (Released)">
            </div>
            <div>
                <h2>Generated Image (Test Version)</h2>
                <img src="{test_image_path}" alt="Generated Image (Test)">
            </div>
        </div>
    </body>
    </html>
    """
    html_file_path = f'./generated_folder/comparison/{face_name}_comparison.html'
    with open(html_file_path, 'w') as file:
        file.write(html_content)
    # print(f'HTML file created at {html_file_path}')

In [5]:
def main(model_url, face_folder, prompt, age, bodyShape, ethnic, sex, skinColor):

    ensure_directory_exists('./generated_folder')
    face_list = list(Path(face_folder).glob('*.jpg'))

    for face_image_path in tqdm(face_list, total=len(face_list)):
        face_name = face_image_path.stem
        face_image_path = str(face_image_path)
        
        # Send to the released version
        released_image_path = send_image(model_url, face_image_path, 'released', prompt, age, bodyShape, ethnic, sex, skinColor)

        # Send to the test version
        test_image_path = send_image(model_url, face_image_path, 'test', prompt, age, bodyShape, ethnic, sex, skinColor)
    

        if released_image_path and test_image_path:
            face_image_path = f'../../{face_image_path}'
            released_image_path = f'../{str(Path(*Path(released_image_path).parts[-2:]))}'
            test_image_path = f'../{str(Path(*Path(test_image_path).parts[-2:]))}'
            # print(face_image_path, released_image_path, test_image_path)
            create_html(model_url, face_image_path, released_image_path, test_image_path, face_name)
        else:
            print(f'{face_image_path}: Failed to generate comparison due to missing images.')

In [6]:
model_url = 'https://www.forever21.com/dw/image/v2/BFKH_PRD/on/demandware.static/-/Sites-f21-master-catalog/default/dw61c6fc57/1_front_750/01262001-02.jpg?sw=1000&sh=1500'
face_folder = 'face_folder'

prompt = 'street'
age = '20'
bodyShape = 'slim'
ethnic = 'asian'
sex = 'female'
skinColor = 'white'

               
main(model_url, face_folder, prompt, age, bodyShape, ethnic, sex, skinColor)

  0%|          | 0/101 [00:00<?, ?it/s]

  4%|▍         | 4/101 [02:02<49:37, 30.69s/it]


KeyboardInterrupt: 